## **Diffusion Equation**

$$
\frac{d\Sigma}{dt} = \frac{3}{r}\frac{\partial}{\partial r} \left[r^{1/2} \frac{\partial}{\partial r}(\nu \Sigma r^{1/2})\right]
$$

This is evolution equation for the surface density of a geometrically thin disk

Putting this in the form of a diffusion equation:

$$
\frac{\partial f}{\partial t} = D \frac{\partial^2 f}{\partial X^2}
$$
where $X \equiv 2r^{1/2}$, $f \equiv \frac{3}{2}\Sigma X$ and $D = \frac{12\nu}{X^2}$


We need numerical values for $\nu$
$$
\nu = \alpha c_s H \\
H = \frac{c_s}{\Omega} \\
\Omega = \sqrt{\frac{G*M_s}{r^3}} \\
c_s = \sqrt{\frac{k_B * T}{\mu * mH}} \\
mH = \frac{1}{N_A} \\
\mu = 2.3 \\
$$

The Temperature profile is given by:
$$
2 \sigma_b T_{disk}^4 = \frac{9}{4}\Sigma \nu \Omega^2\\
T^3 = \frac{9}{8\sigma_b}\Sigma \frac{k_b \alpha}{\mu mH}\sqrt{\frac{GM}{r^3}}
$$

The Pressure is given by:

$$
P = \frac{c_s^2\Sigma}{H} = c_s\Sigma\Omega
$$


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import pandas as pd
import os

In [9]:
#Natural Constants

# T = 600 # Temperature
mu = 2.2 # mean molar mass in disk
avog = 6.02214 * 10**23 # avogadros number
mH = 1.673534 * 10**-27 # atomic hydrogen mass
kB = 1.380649 * 10**-23 #boltzmann constant
G = 6.6738 * 10**-11 #gravitational constant
Ms = 1.9886 * 10**30 #solar mass
Me = 5.972 * 10 **24 #earth mass
AU = 1.4959787 * 10**11 #astronomical units
yrs2sec = 3.1536 * 10**7 #convert years to seconds
sb = 5.6704*10**-8

In [10]:
#Disk Parameters
alpha1 = 1.0 * 10**-2
alpha2 = 1.0 * 10**-3
alpha3 = 1.0 * 10**-4
alphas = [alpha1, alpha2, alpha3]
rin = 0.05 * AU #inner radius of the disk
rout = 30 * AU #outer radius of the disk
sigma_in  = 2 * 10**5 #boundary condition at t=0
sigma_max = sigma_in*2 #boundary condition at t=final_time
sigma_min = 1 * 10**2
distance = rout - rin #distance from inner radius to outer radius

In [11]:
#File settings

In [12]:
#Temporal discretization
max_years = 1000
dyr = .1
dt = dyr * yrs2sec #timestep
final_time = max_years*yrs2sec + 1 #total diffusion time in seconds

In [13]:
#Spacial discretization

n = 600 #number of steps / number of points to plot
dr = distance/n #distance between each spacial step

In [14]:
#Plotting axes
t = np.arange(0, final_time, dt)
t_plot = []

In [15]:
#Initialize grid spaces

dist = np.empty(n)
Omega = np.empty(n)
nu = np.empty(n)
X = np.empty(n)
D = np.empty(n)
sigma = np.empty(n)
f = np.empty(n)
cs2 = np.empty(n)
temp = np.empty(n)
press = np.empty(n)

In [16]:
#Values that are saved

sigma_evol = []
temp_evol = []
press_evol = []
mass_evol = []

In [17]:
#Boundary Conditions

f_in = 0
f_out = 0

In [18]:
# @numba.njit
def calc_init_params(alpha):
    """
    Calculate the initial parameters values for time t = 0 and changing radius.
    """
    for i in range(n):
        dist[i] = (rin + (rout-rin)*i/(n-1))
        sigma[i] = sigma_in * AU / dist[i]
        if (sigma[i]>sigma_max):
            sigma[i] = sigma_max
        if (dist[i]/AU > 15):
            sigma[i] = sigma_min
        X[i] = 2 * np.sqrt(dist[i])
        Omega[i] = np.sqrt(G * Ms / (dist[i] ** 3))
#         temp[i] = ((9*sigma[i]*kB*alpha*Omega[i])/(8*sb*mu*mH))**(1/float(3))
        temp[i]=100
        cs2[i] = (kB * temp[i])/(mu*mH)
        nu[i] = alpha*cs2[i]/Omega[i]
        D[i] = 12 * nu[i] / (X[i] ** 2)
        f[i] = (1.5 * X[i] * sigma[i])

In [19]:
@numba.njit
def calc_temp(f, alpha):
#     for i in range(n):
# #         temp[i] = ((9*sigma[i]*kB*alpha*Omega[i])/(8*sb*mu*mH))**(1/float(3))
#         temp[i]=100
    temp = [100 for i in range(n)]
    return temp

In [20]:
@numba.njit
def calc_press(f, alpha, T):
    """Calculate the pressure evolution at each dt at each  r"""
#     for j in range(n):
#         cs2[j] = (kB * temp[j])/(mu*mH)
#         press[j] = np.sqrt(cs2[j])*sigma[j]*Omega[j]
    press = [np.sqrt((kB * temp[j])/(mu*mH))*sigma[j]*Omega[j] for j in range(n)]
    return press

In [21]:
@numba.njit
def calc_sigma_evol(f, alpha, dt):
    """Outputs the surface density at a specific dt."""
    df_dt = np.empty(n)
    for j in range(1, n-1):
        dX1 = X[j] - X[j-1]
        dX2 = X[j+1] - X[j]
        D1 = 0.5 * (D[j] + D[j-1])
        D2 = 0.5 * (D[j+1] + D[j])
        df_dt[j] = D1 * ((-(f[j] - f[j-1])/dX1**2)) + D2 * ((f[j+1]-f[j])/dX2**2)
    dX_final = X[-1]-X[-2]
    dX_in = X[1]-X[0]
    df_dt[0] = D[0] * (-(f[0] - f_in)/dX_in**2 + (f[1]-f[0])/dX_in**2)
    df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
    f_new = f + df_dt * dt
    return f_new

In [22]:
# @numba.njit
def calc_disk_mass(sigma):
    """Calculate the mass at a specific dt"""
#     sigma_evol = calc_time_evol(f, alpha, dt)
    disk_mass = 0
    for j in range(n):
        disk_mass += 2 * np.pi * dist[j] * dr * sigma[j] #mass of disk at t=dt
#     print(f'The mass of the disk is {disk_mass}.')
    return disk_mass

In [23]:
@numba.njit
def convert_f2sigma(f_new):
    sigma_at_time = [2*f_new[k]/(3*X[k]) for k in range(n)]
    return sigma_at_time

In [24]:
def save2dir(alpha_run):
    if(alpha_run == alpha1):
        output_dir = 'output_a1/'
        filename = 'output_a1/disk_'
    elif(alpha_run == alpha2):
        output_dir = 'output_a2/'
        filename = 'output_a2/disk_'
    else:
        output_dir = 'output_a3/'
        filename = 'output_a3/disk_'
    return (output_dir, filename)

In [25]:
#Time Evolution Main
t_plot_interval = 10 #every ten years
alpha_run = alpha2 #run for different alphas and plot results
output_dir, filename = save2dir(alpha_run)
calc_init_params(alpha_run)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# fig, ax = plt.subplots(4,1)
%matplotlib
for i in range(0, len(t)):
    f_new = calc_sigma_evol(f, alpha_run, dt)
    T = calc_temp(f_new, alpha_run)
    P = calc_press(f_new, T, alpha_run)
    sigma = convert_f2sigma(f_new)
    f = f_new
    if (i%(t_plot_interval/dyr) == 0):
        output = np.vstack([dist/AU, sigma, T, P])
        output = np.transpose(output)
        np.savetxt(filename + str(int(i*dyr)) + '.txt', output, delimiter=',', newline='\n')# if you want to save results to file
        sigma_evol.append(sigma)
        temp_evol.append(T)
        press_evol.append(P)
        mass = calc_disk_mass(sigma)
        mass_evol.append(mass)
        t_plot.append(i*dyr)
sigma_evol = np.array(sigma_evol)
temp_evol = np.array(temp_evol)
press_evol = np.array(press_evol)
for j in range(len(t_plot)):
    plt.title(f'Surface Density at alpha = {alpha_run}')
    plt.xlabel('Distance (AU)')
    plt.ylabel('Surface Density')
    plt.yscale('log')
    plt.plot(dist/AU, sigma_evol[j,:])
    plt.show()
#     ax[0].semilogy(dist/AU, sigma_evol[j,:])
#     ax[1].semilogy(dist/AU, temp_evol[j,:])
#     ax[2].semilogy(dist/AU, press_evol[j,:])
#     # plot mass evolution
#     ax[3].semilogy(t_plot, mass_evol)


Using matplotlib backend: MacOSX


/Users/Rohan/opt/anaconda3/lib/python3.8/site-packages/numba/core/ir_utils.py:2119: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'alpha' of function 'calc_press'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-20-d7eca985dbdb>", line 2:
@numba.njit
def calc_press(f, alpha, T):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [26]:
# @numba.njit
def compare_code(f):
    """Compare C++ code to Python code"""
    telapsed = [100, 1000]
    count = 0
    for time in telapsed:
        df = pd.read_csv('case/radial_' + str(time) + '.txt', delimiter='\t', header=None)
        sigma_c = df[1].to_numpy().astype(np.int)
        sigma_py = sigma_evol[int((time/t_plot_interval)),:]
        plt.title(f'Python vs C++ Surface Density at t={time} years')
        plt.xlabel('Radius (AU)')
        plt.ylabel('Surface Density')
        plt.yscale('log')
        plt.plot(dist/AU, sigma_py, label='Python')
        plt.plot(dist/AU, sigma_c, label='C++')
        plt.legend()
        plt.show()

In [27]:
# @numba.njit
def plot_rms(f):
    """Plot RMS of Python vs C++"""
    t_elapsed = [100, 1000]
    t_rms = np.arange(0, final_time, n)
    rms = []
    for time in t_elapsed:
        df = pd.read_csv('case/radial_' + str(time) + '.txt', delimiter='\t', header=None)
        sigma_c = df[1].to_numpy().astype(np.int)
        sigma_py = sigma_evol[int((time/t_plot_interval)),:]
        rms.append(np.sum(((sigma_c - sigma_py)/(sigma_c))**2))
    plt.title('Root Mean Surface Density at t={time} years')
    plt.xlabel('Time (years)')
    plt.ylabel('RMS')
    plt.yscale('log')
    plt.plot(t_elapsed, rms)
    plt.show()